In [1]:
import numpy as np
import json 


In [2]:
import torch
import requests
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16, device_map=0)



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
img_url = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/demo.jpg' 
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert('RGB')
#raw_image.save("demo.jpg")

question = "Question: Is there a woman in this picture? answer:"
inputs = processor(raw_image, question, return_tensors="pt").to("cuda:0", torch.float16)

out = model.generate(**inputs)
print(out)
print(processor.decode(out[0], skip_special_tokens=True).strip())

/home/tlips/.conda/envs/dsd/lib/python3.10/site-packages/transformers/generation/utils.py:1128: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


tensor([[    2,   117,     6,    89,    16,   117,   693,    11,    42,  2170,
         50118]], device='cuda:0')
no, there is no woman in this picture


In [4]:
def ask_question(prompt):
    inputs = processor(raw_image, prompt, return_tensors="pt").to("cuda:0", torch.float16)
    out = model.generate(**inputs, max_new_tokens=100,do_sample=True)
    return processor.decode(out[0], skip_special_tokens=True).strip()

In [5]:

ask_question("Is there a woman in this picture?")

'Or is that a Labrador Retriever?'

In [6]:
ask_question("Question: Is there a woman in this picture? Answer:")

'There are many questions about the origins of the word "dog"'

In [7]:
ask_question("Question: Is there a woman in this picture? answer:")


'no'

In [8]:
raw_image = Image.open("/home/tlips/Documents/diffusing-synthetic-data/data/real/shoes/dsd-shoes-real_resized_512x512/images/IMG20240516095241.jpg")

In [9]:
from transformers import set_seed
set_seed(4)

In [16]:
ask_question("Question: Describe this image, including a detailed description of the environment. Answer:")

'a man is putting a shoe in a blue cup and it is on a wooden table'